In [52]:
# from google.colab import drive
# drive.mount('/content/drive/')

# 이론 (아래에 빈칸에 답을 서술하시오)

1. 이름 = {철수, 영희, 가영, 나영, 단군}, 성별 = {남, 여}, 나이 = {19, 20, 29, 39, 5000}
2. (철수, 남), (영희, 여), (가영, 여), (나영, 여), (단군, 남), (영희, 남), Dom(R) = {철수, 영희, 가영, 나영, 단군}, Im(R) = {남, 여}
3. 함수가 아니다. Dom(R) = 이름 은 만족시키지만, (영희, 여), (영희, 남) 모두 이 관계에 포함되면서 남 != 여 이기 때문에 함수가 아니다.

# 실습: approximate function

In [53]:
import IPython.display as ipd
import librosa
import librosa.display
import numpy as np

# filename = "/content/drive/MyDrive/ColabNotebooks/sound.wav" # google colab 용
filename = "sound.wav" 
data,sample_rate = librosa.load(filename)
ipd.Audio(filename)

In [54]:
print('data :', data[:5])
print('data size : ', len(data))

data : [0.         0.02578735 0.05151367 0.0770874  0.10244751]
data size :  11025


In [115]:
cf = [0, 0, 0, 0 ,0]

def func(cf) :
    time = np.linspace(0, 0.5, len(data))
    sound = np.sum(cf[0]*np.sin(np.array( [i*time for i in cf[1:]] )), axis=0)
    return sound

def mse(func):  # actually, this is MAE(Mean Absolute Error)
    error=np.abs(func()-data)
    return np.mean(error)*1500

def test() :
    return func(cf)

print('func error : ', mse(test))

func error :  211.1571269132653


In [229]:
# 여기서부터 코드를 작성하시오

init_cf = [max(data) / 4, 800, 1000, 1200, 1400]
# init_cf = [0., 0, 0, 0, 0]

# best_cf를 찾으시오

##################
# first attempt
best_cf = init_cf
err = 1000000000    # 초기 에러값
eps = 100   # 탐색 범위
eps2 = 1.1
trial = 160 # 시도횟수

for i in range(trial):
    c_t = np.array(best_cf.copy()) - eps
    c_t[0] += eps - (eps / 1000)

    for j in range(0, 5):
        eps_t = eps
        if j == 0:
            eps_t /= 10
        
        # eps를 줄여가며 best_cf 각 요소 +- eps 범위를 eps / 10 만큼 증가시키며 탐색
        for c in np.arange(0, eps_t * 2, eps_t / 10):
            cf = best_cf.copy()
            cf[j] = c_t[j] + c
            tmp = mse(test)
            if tmp < err:
                best_cf = cf
                err = tmp
    if i % 20 == 0 :
        print("trial " + str(i) + " : " + str(eps))
        print(err, best_cf)
    eps /= eps2

cf = best_cf

err = mse(test)
print("\nfinal : " + str(err) + ", " + str(best_cf))
if err <= 5:
    print("pass the test")
else:
    print("fail the test")
##################

trial 0 : 100
187.3596405865512 [0.023023986816411934, 820.0, 1040.0, 1230.0, 1460.0]
trial 20 : 14.864362802414345
63.50427333511525 [0.15188314795054403, 823.5494539089852, 1036.7772884232893, 1234.3618126321162, 1462.3013112027675]
trial 40 : 2.209492815217993
6.407257414601961 [0.12387600068485627, 822.7459399634503, 1036.1347643902236, 1233.6534691428012, 1461.6583489827726]
trial 60 : 0.3284270281472808
2.097594572124425 [0.12387600068485627, 822.8389378613738, 1036.0350054273406, 1233.7875670409326, 1461.5572859159738]
trial 80 : 0.04881858500499931
0.16023244492833794 [0.12493988483745354, 822.8596877013264, 1036.0231305931002, 1233.7756798018895, 1461.5405827423822]
trial 100 : 0.007256571590148145
0.031064355336776962 [0.12500182620688985, 822.8573941759118, 1036.0211464859378, 1233.7736956947272, 1461.5430497744896]
trial 120 : 0.0010786431281765482
0.016442643329430937 [0.12498923215702264, 822.857037952347, 1036.0208875818646, 1233.7734212363832, 1461.5426865663567]
trial 

In [240]:
# 여기서부터 코드를 작성하시오

# init_cf = [max(data) / 4, 800, 1000, 1200, 1400]
init_cf = [max(data) / 4 + 0.01, 820, 1030, 1230, 1460]
# init_cf = [max(data) / 4 + 0.01, 800, 1000, 1200, 1400]

##################
# second attempt

best_cf = init_cf

time = np.linspace(0, 0.5, len(data))

def derivative_1():
    return np.mean(np.sin(np.array([c * time for c in best_cf[1:]])) * np.sign(loss())) * 1500


def derivative_2(c):
    return np.mean(best_cf[0] * time * np.cos(c * time) * np.sign(loss())) * 1500


def loss():
    return np.sum(best_cf[0] * np.sin(np.array( [i * time for i in best_cf[1:]] )), axis=0) - data


attempt = 100000
err = 0.0158
learning_rate = [0.000000001, 0.00001]
# learning_rate = 0.00001
# learning_rate = [0.001, 0.001]

for i in range(attempt):
    cf = [
        best_cf[0] - learning_rate[0] * derivative_1(),
        best_cf[1] - learning_rate[1] * derivative_2(best_cf[1]),
        best_cf[2] - learning_rate[1] * derivative_2(best_cf[2]),
        best_cf[3] - learning_rate[1] * derivative_2(best_cf[3]),
        best_cf[4] - learning_rate[1] * derivative_2(best_cf[4])
        # best_cf[0] - learning_rate * derivative_1(),
        # best_cf[1] - learning_rate * derivative_2(best_cf[1]),
        # best_cf[2] - learning_rate * derivative_2(best_cf[2]),
        # best_cf[3] - learning_rate * derivative_2(best_cf[3]),
        # best_cf[4] - learning_rate * derivative_2(best_cf[4])
    ]
    best_cf = cf
    tmp = mse(test)
    if tmp < err:
        break

    if i % 500 == 0:
        print(f"attempt : {i}, err : {tmp}, cf : {cf}")

cf = best_cf

err = mse(test)
print("\nfinal : " + str(err) + ", " + str(best_cf))
if err <= 5:
    print("pass the test")
else:
    print("fail the test")
##################

attempt : 0, err : 188.4970204365506, cf : [0.13057756270179943, 820.000076471583, 1030.0001419900127, 1230.000098169777, 1460.0000429219924]
attempt : 200, err : 167.67814231725504, cf : [0.08071846852101294, 820.012567135454, 1030.0196296883746, 1230.0143426081445, 1460.0084102626865]
attempt : 400, err : 167.31958275092705, cf : [0.08093704411396935, 820.0246908259325, 1030.037594666495, 1230.0276957574827, 1460.0167071078379]
attempt : 600, err : 166.95757990290298, cf : [0.08127203749325551, 820.0368013668192, 1030.05569178284, 1230.0411369596236, 1460.0249921890281]
attempt : 800, err : 166.59217193878519, cf : [0.08162854780776138, 820.0489439786099, 1030.0739126134074, 1230.0546156215526, 1460.0333065461311]
attempt : 1000, err : 166.22352616489258, cf : [0.0818399045250667, 820.0610645700636, 1030.0923193593553, 1230.0681272043869, 1460.0415986768064]
attempt : 1200, err : 165.85108223544034, cf : [0.08216158850445798, 820.0731981018419, 1030.1108608974885, 1230.0817283539411,

KeyboardInterrupt: 